In [17]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [18]:

pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Optical flow parameters
lk_params = dict(winSize=(20, 20),
                 maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

cap = cv2.VideoCapture(0)

# Initialize variables
old_frame = None
old_gray = None
p0 = None


while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    try:
        landmarks = results.pose_landmarks.landmark
        landmark_array = np.array([[lm.x, lm.y] for lm in landmarks])

        if old_frame is None:
            #Initialize points for Lucas-Kanade optical flow
            old_frame = frame.copy()
            old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
            p0 = np.expand_dims(landmark_array, axis=1).astype(np.float32)

            #Perform Lucas-Kanade optical flow
            frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

            # Select good points
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Update old frame and points
            old_frame = frame.copy()
            old_gray = frame_gray.copy()
            p0 = good_new.reshape(-1, 1, 2)

            # Draw tracks
            for (new, old) in zip(good_new, good_old):
                a, b = new.ravel().astype(int)
                c, d = old.ravel().astype(int)
                cv2.line(frame, (a, b), (c, d), (0, 255, 0), 2)
                cv2.circle(frame, (a, b), 5, (0, 0, 255), -1)
    except:
        pass

    # Render detections
    mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=6, circle_radius=6),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=6, circle_radius=6))

    cv2.imshow('LKMP Feed', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()